# House of Gucci (Market Analysis)

### The goal of this project is to capture the optimal target audience for the movie 'House of Gucci':
- This notebook uses audience movie ratings for a list of 288 movies and demographic data corresponding to 210 cities in the United States.
- First, we find the top 10-similar movies to House of Gucci out of 288 movies based on similar movie tags in the content_metadata_tags dataset.
- Next, we find the top 5 geographic locations by market share for each of the 10 similar movies and obtain a list of average demographic data across each geographic location for each movie.
- Finally, we aggregate the top 10 movies' demographic data to determine the optimal set of demographics for the House of Gucci audience.

In [1]:
'''Imports Libraries and Data.'''

import pandas as pd
import numpy as np
from statistics import mode
import warnings

warnings.filterwarnings('ignore')

# Shows all columns in DataFrame
pd.set_option('max_columns', None)

# Imports data
demographics = pd.read_csv('Data/demographics.csv')
dma_market_share = pd.read_csv('Data/dma_market_share.csv')
house_of_gucci_tags = set(pd.read_csv('Data/house_of_gucci_tags.csv')['House of Gucci'].values)
content_metadata_tags = pd.read_csv('Data/content_metadata_tags.csv').drop(['US Release'], axis=1)

### Data Preprocessing

In [2]:
'''Converts Numeric Columns in String Form to Float.'''

for col in demographics.columns:
    if isinstance(demographics.at[0, col], str) and not col in ['DMA', 'DMA Name', 'Metric Type', 'DMA + Metric Type']:
        demographics[col] = demographics[col].str.replace(',', '')
        demographics[col] = demographics[col].str.replace('$', '')
        demographics[col] = demographics[col].astype(float)

In [3]:
'''Excludes Canada and Superfluous Values From dma_market_share.'''

dma_market_share.loc[dma_market_share['Film Share (All Weeks)'] == '-']
dma_market_share = dma_market_share.loc[~((dma_market_share['Film Share (All Weeks)'] == '-') | (dma_market_share['Index'] == '#VALUE!') | (dma_market_share['Index'] == '#DIV/0!'))]
dma_market_share = dma_market_share.loc[~(dma_market_share['DMA Code'] == 'NA: Canada')]

# Converts columns in dma_market_share to float
dma_market_share['Typical Market Share'] = dma_market_share['Typical Market Share'].str.replace('%', '').astype(float)
dma_market_share['Film Share (All Weeks)'] = dma_market_share['Film Share (All Weeks)'].str.replace('%', '').astype(float)
dma_market_share['Index'] = dma_market_share['Index'].str.replace('%', '').astype(float)

### Finding All Existing Tags Within 288 Movies

In [4]:
'''Gets Intersection of Assignment 2 and Assignment 3's Movies and Stores in DataFrame.'''

# Drops unnamed columns
demographics = demographics.drop([i for i in demographics.columns if i[:7] == 'Unnamed'], axis=1)

# Gets unique film lists
unique_films = list(dma_market_share.Film.unique())
assignment_2_movies = list(content_metadata_tags.Name.unique())

# Finds movie overlap in lists
intersection_movies = list(set(unique_films).intersection(set(assignment_2_movies)))
intersection_df = content_metadata_tags.loc[content_metadata_tags['Name'].isin(intersection_movies)]
intersection_df = intersection_df.reset_index(drop=True)

# Changes index to movie names, transposes, and removes superfluous values
intersection_df.index = list(intersection_df.Name.values)
intersection_df = intersection_df.drop(['Name'], axis=1).T
intersection_df = intersection_df[(intersection_df.index.str.contains('_KW')) &
                                  ~(intersection_df.index.str.contains('(character)')) & 
                                  ~(intersection_df.index.str.contains('(franchise)'))]

In [5]:
'''Creates DataFrame With Movies' Tags.'''

film_tags = pd.DataFrame()
for col in intersection_df.columns:
    tags_i = list(intersection_df.loc[intersection_df[col] == 1][col].keys())
    
    for i, t in enumerate(tags_i):
        film_tags.at[i, col] = t
        
film_tags.head()

,10 Cloverfield Lane,12 Strong,About Last Night,Abraham Lincoln: Vampire Hunter,Act of Valor,Action Point,Addicted,Admission,Adrift,Alien: Covenant,Allied,Almost Christmas,American Hustle,American Made,American Ultra,Annabelle,Annabelle: Creation,Annihilation,Ant-Man,Argo,Arrival,Assassin's Creed,Atomic Blonde,Avengers: Age Of Ultron,Avengers: Infinity War,Bad Moms,Bad Samaritan,Baggage Claim,Barbershop: The Next Cut,Battle of the Sexes,Beauty and the Beast,Begin Again,Beirut,Big Hero 6,Black Mass,Black Panther,Blade Runner 2049,Blair Witch,Blended,Blockers,Book Club,Breaking In,Brick Mansions,Bridget Jones's Baby,Broken City,Burnt,Captain America: Civil War,Captain Underpants: The First Epic Movie,Cars 3,Chappaquiddick,Creed,Crimson Peak,Daddy's Home,Daddy's Home 2,Deadpool,Deadpool 2,Death Wish,Den of Thieves,Despicable Me 3,Detroit,Dirty Grandpa,Do You Believe?,Doctor Strange,Dolphin Tale 2,Don't Breathe,Dope,Downsizing,Dracula Untold,Dunkirk,Early Man,Elysium,Entourage,Epic,Escape From Planet Earth,Every Day,Everybody Wants Some!!,"Everything, Everything",Fantastic Four,Fences,Ferdinand,Fifty Shades Darker,Fifty Shades Freed,Finding Dory,Fist Fight,Flatliners,Florence Foster Jenkins,Focus,Frankenweenie,Free Birds,Fruitvale Station,Furious 7,Fury,Game Night,Geostorm,Get On Up,Get Out,Ghost in the Shell,Gifted,Girls Trip,God's Not Dead,God's Not Dead 2,Goosebumps,Gravity,Gringo,Grown Ups 2,Grudge Match,Hacksaw Ridge,Happy Death Day,Hereditary,Hitman: Agent 47,Home,Home Again,Hope Springs,Hostiles,Hot Tub Time Machine 2,Hotel Artemis,I Can Only Imagine,I Feel Pretty,"I, Tonya",Ice Age: Collision Course,Identity Thief,If I Stay,Inferno,Inside Out,Insidious: The Last Key,Interstellar,It,It Comes At Night,Jack Reacher,Jack Reacher: Never Go Back,Jack Ryan: Shadow Recruit,Jackass Presents: Bad Grandpa,Jackie,Jason Bourne,Jersey Boys,Jigsaw,John Wick,Joy,Jumanji: Welcome to the Jungle,Jurassic World,Justice League,Keanu,Kidnap,Kong: Skull Island,Krampus,La La Land,Last Vegas,Leap!,Lee Daniels' The Butler,Let's Be Cops,Life,Lights Out,Logan,Logan Lucky,London Has Fallen,Lone Survivor,"Love, Simon",Lucy,Mad Max: Fury Road,Magic Mike,Magic Mike XXL,Maleficent,Marshall,Max,Maze Runner: The Death Cure,Me Before You,Mechanic: Resurrection,Megan Leavey,Midnight Sun,Million Dollar Arm,Miracles From Heaven,Moana,Molly's Game,Monster Trucks,Moonlight,My Big Fat Greek Wedding 2,Neighbors,Neighbors 2: Sorority Rising,Nine Lives,No Good Deed,Nocturnal Animals,Non-Stop,Ocean's 8,Oculus,Overboard,Pacific Rim,Paddington,Paddington 2,Paper Towns,Paranormal Activity: The Ghost Dimension,Paranormal Activity: The Marked Ones,ParaNorman,Parental Guidance,Peter Rabbit,Pete's Dragon,Pitch Perfect 3,Pixels,Planes,Planes: Fire and Rescue,Poltergeist,Project Almanac,Project X,Prometheus,Proud Mary,Rampage,Ready Player One,Red Sparrow,Red Tails,Resident Evil: Retribution,Resident Evil: The Final Chapter,Ride Along,Ride Along 2,Rings,Rise of the Guardians,Risen,Rogue One: A Star Wars Story,Rough Night,Run All Night,Rush,Safe Haven,San Andreas,Savages,Saving Mr. Banks,Self/Less,Selma,Sgt. Stubby: An American Hero,Sherlock Gnomes,Show Dogs,Sicario,Sing,Sinister,Sinister 2,Skyfall,Sleepless,Smurfs: The Lost Village,Snowden,Solo: A Star Wars Story,Southpaw,Spectre,Split,Spotlight,Spy,Star Wars: The Force Awakens,Star Wars: The Last Jedi,Steve Jobs,Storks,Suicide Squad,Sully,Taken 2,Taken 3,Tammy,Thank You For Your Service,The Wedding Ringer,This Is 40,This Is The End,Thor: Ragnarok,Tomb Raider,Tomorrowland,Transcendence,Transformers: Age of Extinction,Transformers: The Last Knight,Trolls,Tully,Turbo,Tyler Perry's Acrimony,Unbroken,Underworld: Blood Wars,Unforgettable,Unfriended,Unsane,Upgrade,War Room,Warcraft,What To Expect When You're Expecting,Whiskey Tango Foxtrot,Why Him?,Wild,Winchester,Wind River,Wish Upon,Wonder,Wonder Woman,World War Z,Wreck-It Ralph,X-Men: Apocalypse,Zootopia
0,2010s_KW,IMAX_KW,2010s_KW,1800s_KW,terrorism_KW,2010s_KW,2010s_KW,high scho

### Finds Overlap of 288 Movies' and House of Gucci's Tags

- Sorts Columns by Most-Similar Movies

In [6]:
'''Creates Dictionary with Each Film's Tag Overlap with House of Gucci.'''

film_ratings = {}
for col in film_tags.columns:
    film_i_movies = set(film_tags[col].values)
    overlap = house_of_gucci_tags.intersection(film_i_movies)
    
    film_ratings[col] = [len(overlap), list(overlap)]

In [7]:
'''Creates DataFrame with Overlapping Tags.'''

overlap_df = pd.DataFrame()
for film in film_ratings.keys():
    tags_list_len = str(film_ratings[film][0])
    movies = film_ratings[film][1]
    
    overlap_df.at['length', film] = tags_list_len
    
    for t in range(int(tags_list_len)):
        overlap_df.at[t+1, film] = movies[t]
        
# Sorts by length of movie overlap list
overlap_df = overlap_df.T
overlap_df['length'] = overlap_df['length'].astype(int)
overlap_df = overlap_df.sort_values(by=['length'], ascending=False)
overlap_df = overlap_df.T

In [8]:
overlap_df.head()

,"I, Tonya",Molly's Game,Creed,Black Mass,Taken 3,Hereditary,Star Wars: The Force Awakens,Jackie,Saving Mr. Banks,Moonlight,Tully,Gifted,Jack Ryan: Shadow Recruit,Rush,Run All Night,Red Sparrow,Steve Jobs,Unforgettable,Fruitvale Station,Joy,About Last Night,This Is 40,Blended,Daddy's Home,American Hustle,Wild,Nocturnal Animals,Almost Christmas,Addicted,Battle of the Sexes,Book Club,Self/Less,Florence Foster Jenkins,Ride Along,Death Wish,Bridget Jones's Baby,Non-Stop,Nine Lives,Beauty and the Beast,Neighbors 2: Sorority Rising,Flatliners,Southpaw,San Andreas,La La Land,"Everything, Everything",Adrift,Spy,Rogue One: A Star Wars Story,Skyfall,Oculus,Dirty Grandpa,Winchester,Tyler Perry's Acrimony,Assassin's Creed,Baggage Claim,Gravity,Bad Moms,Tammy,Fences,Wish Upon,Proud Mary,Taken 2,Last Vegas,Daddy's Home 2,Interstellar,Breaking In,Spotlight,Action Point,Blockers,If I Stay,Star Wars: The Last Jedi,Tomb Raider,John Wick,Tomorrowland,Get On Up,Midnight Sun,Chappaquiddick,My Big Fat Greek Wedding 2,Max,Sinister 2,Atomic Blonde,Argo,Me Before You,Marshall,Sicario,Grudge Match,Girls Trip,Get Out,Focus,Megan Leavey,Paranormal Activity: The Ghost Dimension,Wonder,Admission,Allied,Poltergeist,Whiskey Tango Foxtrot,No Good Deed,Begin Again,Neighbors,Beirut,Spectre,Magic Mike,Transcendence,Epic,Rough Night,Maleficent,Jigsaw,Safe Haven,Lights Out,Snowden,Million Dollar Arm,Paddington 2,Sleepless,Pacific Rim,Jersey Boys,Kidnap,Pete's Dragon,10 Cloverfield Lane,Game Night,War Room,Dracula Untold,Escape From Planet Earth,Dolphin Tale 2,Detroit,The Wedding Ringer,Thank You For Your Service,Unsane,Sully,Brick Mansions,Upgrade,I Can Only Imagine,Black Panther,Wind River,Barbershop: The Next Cut,Why Him?,Arrival,Parental Guidance,Lucy,Mad Max: Fury Road,American Made,Don't Breathe,Ride Along 2,American Ultra,Annabelle,Underworld: Blood Wars,Unbroken,Warcraft,Deadpool,Crimson Peak,Annihilation,Unfriended,"Love, Simon",Burnt,Broken City,I Feel Pretty,London Has Fallen,Transformers: Age of Extinction,Dunkirk,Identity Thief,Split,Inferno,Hot Tub Time Machine 2,Hope Springs,Home Again,Hitman: Agent 47,It Comes At Night,Happy Death Day,Hacksaw Ridge,Goosebumps,Jackass Presents: Bad Grandpa,Suicide Squad,Ghost in the Shell,Paranormal Activity: The Marked Ones,ParaNorman,Fifty Shades Darker,Everybody Wants Some!!,Lee Daniels' The Butler,Leap!,Geostorm,Krampus,Fantastic Four,Fifty Shades Freed,Finding Dory,Selma,Turbo,Planes: Fire and Rescue,Sgt. Stubby: An American Hero,Ready Player One,This Is The End,Wonder Woman,Prometheus,Kong: Skull Island,Paper Towns,Ferdinand,It,Grown Ups 2,Fury,Furious 7,Free Birds,Fist Fight,Every Day,Logan Lucky,Downsizing,Despicable Me 3,Captain America: Civil War,Blair Witch,Blade Runner 2049,Bad Samaritan,Jack Reacher: Never Go Back,Hotel Artemis,Mechanic: Resurrection,Miracles From Heaven,Moana,Paddington,Ocean's 8,Den of Thieves,Gringo,God's Not Dead 2,Storks,God's Not Dead,Magic Mike XXL,Project Almanac,Elysium,Trolls,Overboard,Maze Runner: The Death Cure,Hostiles,Planes,Pixels,What To Expect When You're Expecting,Avengers: Infinity War,Ant-Man,Annabelle: Creation,Alien: Covenant,Abraham Lincoln: Vampire Hunter,World War Z,Wreck-It Ralph,X-Men: Apocalypse,Cars 3,Zootopia,Sinister,Keanu,Jason Bourne,12 Strong,Sing,Let's Be Cops,Jurassic World,Life,Logan,Resident Evil: The Final Chapter,Jack Reacher,Lone Survivor,Risen,Insidious: The Last Key,Inside Out,Rings,Show Dogs,Big Hero 6,Pitch Perfect 3,Justice League,Peter Rabbit,Savages,Avengers: Age Of Ultron,Act of Valor,Rise of the Guardians,Captain Underpants: The First Epic Movie,Do You Believe?,Deadpool 2,Project X,Ice Age: Collision Course,Solo: A Star Wars Story,Smurfs: The Lost Village,Red Tails,Resident Evil: Retribution,Frankenweenie,Monster Trucks,Entourage,Jumanji: Welcome to the Jungle,Thor: Ragnarok,Early Man,Dope,Transformers: The Last Knight,Doctor Strange,Home,Sherlock Gnomes,Rampage
length,17,13,12,12,11,11,11,11,11,11,11,11,11,11,10,10,10,10,10,10,10,9,9,9,9

### Capturing House of Gucci's Demographics

In [9]:
dma_market_share.head()

,Rank,DMA Long Name,DMA,DMA Code,Typical Market Share,Film Share (All Weeks),Index,Film
0,1,Los Angeles (2),Los Angeles,803,8.19,7.42,-9.51,Jumanji: Welcome to the Jungle
1,2,New York (1),New York,501,6.41,4.96,-22.59,Jumanji: Welcome to the Jungle
2,3,Dallas-Ft. Worth (5),Dallas-Ft. Worth,623,2.77,2.79,0.86,Jumanji: Welcome to the Jungle
3,4,San Francisco-Oak-San Jose (6),San Francisco-Oak-San Jose,807,3.07,2.73,-11.09,Jumanji: Welcome to the Jungle
4,5,Chicago (3),Chicago,602,2.93,2.68,-8.54,Jumanji: Welcome to the Jungle


In [10]:
'''Gets Top 5 DMAs for Each of the Top 10-Similar Movies.'''

# Filters dataframe to top 10 movies
top_10_movies_list = ['I, Tonya', "Molly's Game", 'Creed', 'Black Mass', 'Taken 3', 'Hereditary',
                      'Star Wars: The Force Awakens', 'Jackie', 'Saving Mr. Banks', 'Moonlight']
top_10_movies = dma_market_share.loc[dma_market_share['Film'].isin(top_10_movies_list)]

# Gets top 5 DMAs
top_dma = {}
for movie in top_10_movies_list:
    temp_df = top_10_movies.loc[top_10_movies['Film'] == movie]
    top_dma[movie] = list(temp_df.sort_values(['Index'], ascending=False)['DMA Code'].values[:5])

In [11]:
'''Gathers Top 2 Values Per Demographic For Each Similar Movie.'''

top_10_movies_demos = pd.DataFrame()
for movie in top_10_movies_list:
    top_pops, top_lang, top_gend, top_age, top_married, top_education, top_income, top_family, top_occ, top_occ_cl = [], [], [], [], [], [], [], [], [], []

    for dma in top_dma[movie]:
        temp_df = demographics.loc[demographics['DMA Area Code'] == int(dma)]
        
        # Population by Single-Classification Race
        # Population Top 1
        population_cols = ['White Alone', 'Black or African American Alone', 'Amer. Indian and Alaska Native Alone', 'Asian Alone',
                           'Native Hawaiian and Other Pac. Isl. Alone', 'Some Other Race Alone', 'Two or More Races']
        population_1 = temp_df[population_cols].reset_index(drop=True)
        top_pops.append(population_1.idxmax(axis=1).values[0])
        # Population Top 2
        population_cols.remove(population_1.idxmax(axis=1).values[0])
        population_2 = temp_df[population_cols].reset_index(drop=True)
        top_pops.append(population_2.idxmax(axis=1).values[0])
        
        
        # Language
        # Language Top 1
        language_cols = ['Speak Only English at Home', 'Speak Asian/Pac. Isl. Lang. at Home',
                         'Speak IndoEuropean Language at Home', 'Speak Spanish at Home', 'Speak Other Language at Home']
        language_1 = temp_df[language_cols].reset_index(drop=True)
        top_lang.append(language_1.idxmax(axis=1).values[0])
        # Language Top 2
        language_cols.remove(language_1.idxmax(axis=1).values[0])
        language_2 = temp_df[language_cols].reset_index(drop=True)
        top_lang.append(language_2.idxmax(axis=1).values[0])
        
        
        # Gender
        gender_cols = ['Male', 'Female']
        gender_1 = temp_df[gender_cols].reset_index(drop=True)
        top_gend.append(gender_1.idxmax(axis=1).values[0])
        
        
        # Age
        # Age Top 1
        age_cols = ['Age 0 - 4.1', 'Age 5 - 9.1', 'Age 10 - 14.1', 'Age 15 - 17.1', 'Age 18 - 20.1', 'Age 21 - 24.1',
                    'Age 25 - 34.1', 'Age 35 - 44.1', 'Age 45 - 54.1', 'Age 55 - 64.1', 'Age 65 - 74.1', 'Age 75 - 84.1', 'Age 85 and over.1']
        age_1 = temp_df[age_cols].reset_index(drop=True)
        top_age.append(age_1.idxmax(axis=1).values[0])
        # Age Top 2
        age_cols.remove(age_1.idxmax(axis=1).values[0])
        age_2 = temp_df[age_cols].reset_index(drop=True)
        top_age.append(age_2.idxmax(axis=1).values[0])
        
        
        # Marital Status
        # Married Top 1
        married_cols = ['Total, Never Married', 'Married, Spouse present', 'Married, Spouse absent', 'Widowed', 'Divorced']
        married_1 = temp_df[married_cols].reset_index(drop=True)
        top_married.append(married_1.idxmax(axis=1).values[0])
        # Married Top 2
        married_cols.remove(married_1.idxmax(axis=1).values[0])
        married_2 = temp_df[married_cols].reset_index(drop=True)
        top_married.append(married_2.idxmax(axis=1).values[0])
        
        
        # Education
        # Education Top 1
        education_cols = ['Less than 9th grade', 'Some High School, no diploma', 'High School Graduate (or GED)', 'Some College, no degree',
                        'Associate Degree', "Bachelor's Degree", "Master's Degree", 'Professional School Degree', 'Doctorate Degree']
        education_1 = temp_df[education_cols].reset_index(drop=True)
        top_education.append(education_1.idxmax(axis=1).values[0])
        # Education Top 2
        education_cols.remove(education_1.idxmax(axis=1).values[0])
        education_2 = temp_df[education_cols].reset_index(drop=True)
        top_education.append(education_2.idxmax(axis=1).values[0])
        
        
        # Income
        # Income Top 1
        income_cols = ['Income < $15,000', 'Income $15,000 - $24,999', 'Income $25,000 - $34,999', 'Income $35,000 - $49,999', 'Income $50,000 - $74,999',
                       'Income $75,000 - $99,999', 'Income $100,000 - $124,999', 'Income $125,000 - $149,999', 'Income $150,000 - $199,999',
                       'Income $200,000 - $249,999', 'Income $250,000 - $499,999', 'Income $500,000+']
        income_1 = temp_df[income_cols].reset_index(drop=True)
        top_income.append(income_1.idxmax(axis=1).values[0])
        # Income Top 2
        income_cols.remove(income_1.idxmax(axis=1).values[0])
        income_2 = temp_df[income_cols].reset_index(drop=True)
        top_income.append(income_2.idxmax(axis=1).values[0])
        
        
        # Family Type
        # Family Top 1
        family_cols = ['Married-Couple Family, own children', 'Married-Couple Family, no own children', 'Male Householder, own children',
                       'Male Householder, no own children', 'Female Householder, own children', 'Female Householder, no own children']
        family_1 = temp_df[family_cols].reset_index(drop=True)
        top_family.append(family_1.idxmax(axis=1).values[0])
        # Family Top 2
        family_cols.remove(family_1.idxmax(axis=1).values[0])
        family_2 = temp_df[family_cols].reset_index(drop=True)
        top_family.append(family_2.idxmax(axis=1).values[0])
        
        
        # Occupation
        # Occupation Top 1
        occ_cols = ['Architect/Engineer', 'Arts/Entertainment/Sports', 'Building Grounds Maintenance', 'Business/Financial Operations', 'Community/Social Services',
                    'Computer/Mathematical', 'Construction/Extraction', 'Education/Training/Library', 'Farming/Fishing/Forestry', 'Food Prep/Serving',
                    'Health Practitioner/Technician', 'Healthcare Support', 'Maintenance Repair', 'Legal', 'Life/Physical/Social Science', 'Management',
                    'Office/Admin. Support', 'Production', 'Protective Services', 'Sales/Related', 'Personal Care/Service', 'Transportation/Moving']
        occ_1 = temp_df[occ_cols].reset_index(drop=True)
        top_occ.append(occ_1.idxmax(axis=1).values[0])
        # Occupation Top 2
        occ_cols.remove(occ_1.idxmax(axis=1).values[0])
        occ_2 = temp_df[occ_cols].reset_index(drop=True)
        top_occ.append(occ_2.idxmax(axis=1).values[0])
        # Occupation Top 3
        occ_cols.remove(occ_2.idxmax(axis=1).values[0])
        occ_3 = temp_df[occ_cols].reset_index(drop=True)
        top_occ.append(occ_3.idxmax(axis=1).values[0])
        # Occupation Top 4
        occ_cols.remove(occ_3.idxmax(axis=1).values[0])
        occ_4 = temp_df[occ_cols].reset_index(drop=True)
        top_occ.append(occ_4.idxmax(axis=1).values[0])
        # Occupation Top 5
        occ_cols.remove(occ_4.idxmax(axis=1).values[0])
        occ_5 = temp_df[occ_cols].reset_index(drop=True)
        top_occ.append(occ_5.idxmax(axis=1).values[0])
        
        
        # Occupation Class
        occ_cl_cols = ['Blue Collar', 'White Collar', 'Service and Farm']
        occ_cl_1 = temp_df[occ_cl_cols].reset_index(drop=True)
        top_occ_cl.append(occ_cl_1.idxmax(axis=1).values[0])

        
    '''Aggregates Top Demographics Across All 5 DMAs.'''
    # Population Aggregation
    pop_top_1 = mode(top_pops)
    pop_top_2 = mode([i for i in top_pops if i != pop_top_1])
    top_10_movies_demos.at[movie, 'Population'] = ', '.join([pop_top_1, pop_top_2])
    
    # Language Aggregation
    lang_top_1 = mode(top_lang)
    lang_top_2 = mode([i for i in top_lang if i != lang_top_1])
    top_10_movies_demos.at[movie, 'Language'] = ', '.join([lang_top_1, lang_top_2])
    
    # Gender Aggregation
    top_10_movies_demos.at[movie, 'Gender'] = mode(top_gend)
    
    # Age Aggregation
    age_top_1 = mode(top_age)
    age_top_2 = mode([i for i in top_age if i != age_top_1])
    top_10_movies_demos.at[movie, 'Age'] = ', '.join([age_top_1, age_top_2])
    
    # Married Aggregation
    married_top_1 = mode(top_married)
    married_top_2 = mode([i for i in top_married if i != married_top_1])
    top_10_movies_demos.at[movie, 'Marital Status'] = ', '.join([married_top_1, married_top_2])
    
    # Education Aggregation
    education_top_1 = mode(top_education)
    education_top_2 = mode([i for i in top_education if i != education_top_1])
    top_10_movies_demos.at[movie, 'Education'] = ', '.join([education_top_1, education_top_2])
    
    # Income Aggregation
    income_top_1 = mode(top_income)
    income_top_2 = mode([i for i in top_income if i != income_top_1])
    top_10_movies_demos.at[movie, 'Income'] = ', '.join([income_top_1, income_top_2])
    
    # Family Aggregation
    family_top_1 = mode(top_family)
    family_top_2 = mode([i for i in top_family if i != family_top_1])
    top_10_movies_demos.at[movie, 'Family Type'] = ', '.join([family_top_1, family_top_2])
    
    # Occupation Aggregation
    occ_top_1 = mode(top_occ)
    occ_top_2 = mode([i for i in top_occ if i != occ_top_1])
    occ_top_3 = mode([i for i in top_occ if i != occ_top_1 and i != occ_top_2])
    occ_top_4 = mode([i for i in top_occ if i != occ_top_1 and i != occ_top_2 and i != occ_top_3])
    occ_top_5 = mode([i for i in top_occ if i != occ_top_1 and i != occ_top_2 and i != occ_top_3 and i != occ_top_4])
    top_10_movies_demos.at[movie, 'Occupation'] = ', '.join([occ_top_1, occ_top_2, occ_top_3, occ_top_4, occ_top_5])
    
    # Occupation Class Aggregation
    top_10_movies_demos.at[movie, 'Occupation Class'] = mode(top_occ_cl)

In [12]:
top_10_movies_demos.head(3)

,Population,Language,Gender,Age,Marital Status,Education,Income,Family Type,Occupation,Occupation Class
"I, Tonya","White Alone, Amer. Indian and Alaska Native Alone","Speak Only English at Home, Speak IndoEuropean...",Male,"Age 55 - 64.1, Age 45 - 54.1","Married, Spouse present, Total, Never Married","High School Graduate (or GED), Some College, n...","Income $50,000 - $74,999, Income $35,000 - $49...","Married-Couple Family, no own children, Marrie...","Office/Admin. Support, Sales/Related, Manageme...",White Collar
Molly's Game,"White Alone, Black or African American Alone","Speak Only English at Home, Speak Spanish at Home",Female,"Age 25 - 34.1, Age 35 - 44.1","Married, Spouse present, Total, Never Married","High School Graduate (or GED), Some College, n...","Income $50,000 - $74,999, Income $35,000 - $49...","Married-Couple Family, no own children, Marrie...","Office/Admin. Support, Sales/Related, Manageme...",White Collar
Creed,"White Alone, Some Other Race Alone","Speak Only English at Home, Speak Spanish at Home",Female,"Age 25 - 34.1, Age 35 - 44.1","Married, Spouse present, Total, Never Married","High School Graduate (or GED), Some College, n...","Income $50,000 - $74,999, Income < $15,000","Married-Couple Family, no own children, Marrie...","Office/Admin. Support, Sales/Related, Manageme...",White Collar


In [13]:
'''Gets Top Values In Each Demographic Column and Assigns to House of Gucci.'''

house_of_gucci_dem = pd.DataFrame()
for col in top_10_movies_demos.columns:
    total_vals = []
    gets_total_vals = [[total_vals.append(j) for j in i.split(', ')] for i in top_10_movies_demos[col].values]
    
    if col in ['Gender', 'Occupation Class']:
        top_1 = mode(total_vals)
        house_of_gucci_dem.at['House of Gucci', col] = top_1
    elif col == 'Occupation':
        top_1 = mode(total_vals)
        top_2 = mode([i for i in total_vals if i != top_1])
        top_3 = mode([i for i in total_vals if i != top_1 and i != top_2])
        top_4 = mode([i for i in total_vals if i != top_1 and i != top_2 and i != top_3])
        top_5 = mode([i for i in total_vals if i != top_1 and i != top_2 and i != top_3 and i != top_4])
        house_of_gucci_dem.at['House of Gucci', col] = ', '.join([top_1, top_2, top_3, top_4, top_5])
    elif col == 'Population':
        top_1 = mode(total_vals)
        top_2 = mode([i for i in total_vals if i != top_1])
        top_3 = mode([i for i in total_vals if i != top_1 and i != top_2])
        house_of_gucci_dem.at['House of Gucci', col] = ', '.join([top_1, top_2, top_3])
    else:
        top_1 = mode(total_vals)
        top_2 = mode([i for i in total_vals if i != top_1])
        house_of_gucci_dem.at['House of Gucci', col] = ', '.join([top_1, top_2])

# Lists House of Gucci's top demographic data
house_of_gucci_dem

,Population,Language,Gender,Age,Marital Status,Education,Income,Family Type,Occupation,Occupation Class
House of Gucci,"White Alone, Some Other Race Alone, Black or A...","Speak Only English at Home, Speak Spanish at Home",Female,"Age 25 - 34.1, Age 35 - 44.1","Married, Spouse present","High School Graduate (or GED), Some College","Income $50,000 - $74,999, Income $35,000 - $49...","Married-Couple Family, no own children","Office/Admin. Support, Sales/Related, Manageme...",White Collar


In [14]:
'''Lists Top 5 DMAs for House of Gucci.'''

all_dmas = []
gets_all_dmas = [[all_dmas.append(j) for j in i] for i in top_dma.values()]

top_1_dma = mode(all_dmas)
top_2_dma = mode([i for i in all_dmas if i != top_1_dma])
top_3_dma = mode([i for i in all_dmas if i != top_1_dma and i != top_2_dma])
top_4_dma = mode([i for i in all_dmas if i != top_1_dma and i != top_2_dma and i != top_3_dma])
top_5_dma = mode([i for i in all_dmas if i != top_1_dma and i != top_2_dma and i != top_3_dma and i != top_4_dma])

for i, dma in enumerate([top_1_dma, top_2_dma, top_3_dma, top_4_dma, top_5_dma]):
    dma_name = dma_market_share.loc[dma_market_share['DMA Code'] == str(dma)]['DMA'].values[0]
    print(f"DMA {i+1}: {dma_name} ({dma})")

DMA 1: Palm Springs (804)
DMA 2: Helena (766)
DMA 3: West Palm Beach-Ft. Pierce (625)
DMA 4: Charleston, SC (519)
DMA 5: Meridian (711)
